In [1]:
!pip install xgboost

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com


DEPRECATION: Loading egg at d:\summerresearch\summer_research\lib\site-packages\glrm-0.0.1-py3.11.egg is deprecated. pip 25.1 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330

[notice] A new release of pip is available: 24.3.1 -> 25.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import pandas as pd
import numpy as np
import os
import sklearn
import xgboost as xgb
import sys
import matplotlib.pyplot as plt

In [3]:
df_train = pd.read_csv('D:\SummerResearch\data\mnist28\mnist28_train.csv')
df_test = pd.read_csv('D:\SummerResearch\data\mnist28\mnist28_test.csv')

print(df_train.shape)
print(df_test.shape)
df_train.head()


(56000, 785)
(14000, 785)


,0,1,2,3,4,5,6,7,8,9,...,775,776,777,778,779,780,781,782,783,label
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,7
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,9
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [4]:
xgboost = xgb.XGBClassifier()

xgboost.fit(df_train.drop('label', axis=1), df_train['label'])
print("Train score:")
print(xgboost.score(df_train.drop('label', axis=1), df_train['label']))
print("Test score:")
print(xgboost.score(df_test.drop('label', axis=1), df_test['label']))


Train score:
1.0
Test score:
0.9765714285714285


## load trained synthesizer and generate X'

In [5]:
sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), '..')))

from synthesizer import *

synthesizer_file_name = 'D:\SummerResearch\sdv trained model\mnist28\mnist28_synthesizer_onlyX.pkl'
synthesizer = load_synthesizer(synthesizer_file_name)

In [6]:
X_prime_xgb = synthesizer.sample(100000)

NameError: name 'X_prime' is not defined

In [7]:
y_prime_xgb = xgboost.predict(X_prime_xgb)
df_prime_xgb = pd.DataFrame(pd.concat([X_prime_xgb, pd.DataFrame(y_prime_xgb, columns=['label'])], axis=1))

df_prime_xgb.head()

,0,1,2,3,4,5,6,7,8,9,...,775,776,777,778,779,780,781,782,783,label
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,3
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,3
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [8]:
df_prime_xgb.to_csv('D:\SummerResearch\data\mnist28\onehot_mnist28_sdv_xgb_100k.csv', index=False)

## Random Forest

In [9]:
rf = sklearn.ensemble.RandomForestClassifier()
rf.fit(df_train.drop('label', axis=1), df_train['label'])
print("Train score:")
print(rf.score(df_train.drop('label', axis=1), df_train['label']))
print("Test score:")
print(rf.score(df_test.drop('label', axis=1), df_test['label']))

Train score:
1.0
Test score:
0.9681428571428572


In [10]:
X_prime_rf = synthesizer.sample(100000)
y_prime_rf = rf.predict(X_prime_rf)
df_prime_rf = pd.DataFrame(pd.concat([X_prime_rf, pd.DataFrame(y_prime_rf, columns=['label'])], axis=1))
df_prime_rf.head()


,0,1,2,3,4,5,6,7,8,9,...,775,776,777,778,779,780,781,782,783,label
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,3
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,3
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [11]:
df_prime_rf.to_csv('D:\SummerResearch\data\mnist28\onehot_mnist28_sdv_rf_100k.csv', index=False)